### Fastai Classification - Evaluating models on the dataset of Howard et al.
This notebook demonstrates the steps taken to estimate performance of the manufacturer and model classification mode on the additional dataset published alongside the `Cardiac Rhythm Device Identification Using Neural Networks` paper by Howard et al (https://www.jacc.org/doi/10.1016/j.jacep.2019.02.003). To do this, the model/manufacturer composition of the additional dataset is compared against the composition of our dataset. There are 3 common models and 3 common manufacturers between the dataset which are used for testing of our models. Only the test portion of the additional dataset is taken in order to compare the models directly on the same images. 

Since Howard data features images in the form of center crop of devices, segmentation step of our pipeline is irrelevant for experimental purposes, and the images from the addditional dataset are taken as is. 

The mirror experiment, where the model designed by Howard et al is tested using out test data, is available in our paper and has been performed using the web application hosted by the paper authors. 

In [1]:
import torch
import pandas as pd
from fastai.vision.all import *
import os
import numpy as np
import json
import sklearn
import pickle
from pathlib import Path
import warnings
import shutil

warnings.filterwarnings("ignore")

In [2]:
# Setting the paths to the folders with datasets.

abs_dataset_path = Path("/workdir/cied/Dataset")
add_dataset_dir = Path(abs_dataset_path / "202401002_add_dataset")
rel_dataset_path = Path(os.path.relpath(abs_dataset_path))

## Running validation

In [3]:
# Loading the metadata for the additional dataset
add_df = pd.read_excel(add_dataset_dir / "20240203_metadata.xlsx")

In [4]:
# Defining paths to the images according to our scheme
add_df["patch_fname"] = add_df["split"] + "/" + add_df["model"] + "/" + add_df["filename"]
rel_add_dataset_path = Path(os.path.relpath(add_dataset_dir))
add_df.loc[:, "patch_fname"] = add_df.loc[:, "patch_fname"].apply(lambda x: str(rel_add_dataset_path / x))

In [5]:
# Defining the dataset portion used as a validation split used in the original experiments of Howard et al.
add_df["is_valid"] = add_df["split"] == "Test"
add_df = add_df[add_df["is_valid"]]

In [6]:
# Displaying manufacturer information in the dataset. Our dataset has all the manufacturers lists except for SOR (Sorin)
manufacturer_tally = add_df["model"].apply(lambda x: x.split(" ")[0]).value_counts()
manufacturer_tally

MDT    78
BOS    53
SOR    37
STJ    35
BIO    13
Name: model, dtype: int64

In [7]:
# Displaying model information in the dataset. The only common models between the datasets are:
# * Quadra Assura_Unify
# * Allure Quadra
# * REVEAL LINQ

model_tally = add_df["model"].apply(lambda x: x.split("-")[1].lstrip(" ")).value_counts()
model_tally

REVEAL LINQ                                 5
Elect                                       5
C60 DR                                      5
Accent                                      5
EnRhythm                                    5
Emblem                                      5
AT500                                       5
Ovatio                                      5
Ellipse                                     5
Azure                                       5
Allure Quadra                               5
Autogen_Teligen_Energen_Cognis              5
MiniSwing                                   5
Vitality                                    5
Identity                                    5
Ingenio                                     5
Visionist                                   5
Evia                                        5
Reply                                       5
Sigma                                       5
Zephyr                                      5
Neway                             

In [8]:
# Removing the nonmatching models from the dataset
matching_models = [
    "STJ - Quadra Assura_Unify",
    "STJ - Allure Quadra",
    "MDT - REVEAL LINQ",
]

add_df = add_df[add_df["model"].apply(lambda x: x in matching_models)]

In [9]:
# Bringing the column manufacturer column in lime of our schema
add_df.rename(columns={"manufacturer": "Hersteller"}, inplace=True)

In [10]:
# Mapping the model names to our schema
howard_model_mapping = {
    "MDT - REVEAL LINQ": "Reveal LINQ",
    "STJ - Allure Quadra": "Quadra Allure MP",
    "STJ - Quadra Assura_Unify": "Quadra Assura",
}

add_df["Exaktes_Modell"] = add_df["model"].apply(lambda x: howard_model_mapping[x])

In [11]:
# Loading our metadata spreadsheet. This is required, since fastai estimates the
# number of output classes based on the dataloader used when loading the mode

df = pd.read_excel(abs_dataset_path / "04_train_data_clf.xlsx")

# Providing relative paths to the filenames for the dataloader
df.loc[:, "patch_fname"] = df.loc[:, "patch_fname"].apply(lambda x: str(rel_dataset_path / "Classification" / x))

In [12]:
# Adding out train metadata to the additional dataset metadata for reasons stated in the previous cell
holdout_df = add_df.copy()
add_df = pd.concat([df[~df["is_valid"]], add_df])

In [13]:
# Defining the dataloader and loading the model for manufacturer prediction
dls = ImageDataLoaders.from_df(
    add_df,
    fn_col="patch_fname",
    label_col="Hersteller",
    valid_col="is_valid",
    item_tfms=Resize(256),
    batch_tfms=[*aug_transforms(size=256, min_scale=0.1)],
    bs=16,
)

exp_name = "s256_cls_h"

learner = vision_learner(dls, resnet50, metrics=accuracy).to_fp16()
load_model("models/{}".format(exp_name), learner.model, learner.opt, device="cpu")

In [14]:
# Validating the manufacturer prediction model
res = learner.validate()
print(f"Test accuracy: {res[1]:.3}")

Test accuracy: 1.0


In [15]:
# Calculating data for condidence intervals
res = learner.get_preds()

probs = np.array(res[0])
targs = np.array(res[1])
preds = np.argmax(probs, axis=1)
output_df = pd.DataFrame(data=probs, columns=learner.dls[1].vocab)
output_df.columns = [str(i) + " - " + output_df.columns[i] for i in range(len(output_df.columns))]
output_df.loc[:, "Prediction"] = preds
output_df.loc[:, "Target"] = targs
output_df.index = holdout_df["patch_fname"].apply(lambda x: "/".join(x.split("/")[-2:]))
output_df.to_excel(abs_dataset_path / "20240129_preds_confidence_howard_model.xlsx")

In [16]:
# Defining the dataloader and loading the model for model prediction
dls = ImageDataLoaders.from_df(
    add_df,
    fn_col="patch_fname",
    label_col="Exaktes_Modell",
    valid_col="is_valid",
    item_tfms=Resize(256),
    batch_tfms=[*aug_transforms(size=256, min_scale=0.1)],
    bs=16,
)

# Setting the experiment name to save the model
exp_name = "s256_cls_m"

# Loading the model
learner = vision_learner(dls, resnet50, metrics=accuracy).to_fp16()
load_model("models/{}".format(exp_name), learner.model, learner.opt, device="cpu")
# load_model("models/{}".format(exp_name), learner.model, learner.opt, device="cuda:0")

# Estimating model performance on the new data
res = learner.validate()
print(f"Test accuracy: {res[1]:.3}")

Test accuracy: 0.8


In [17]:
# Calculating data for condidence intervals
res = learner.get_preds()

probs = np.array(res[0])
targs = np.array(res[1])
preds = np.argmax(probs, axis=1)
output_df = pd.DataFrame(data=probs, columns=learner.dls[1].vocab)
output_df.columns = [str(i) + " - " + output_df.columns[i] for i in range(len(output_df.columns))]
output_df.loc[:, "Prediction"] = preds
output_df.loc[:, "Target"] = targs
output_df.index = holdout_df["patch_fname"].apply(lambda x: "/".join(x.split("/")[-2:]))
output_df.to_excel(abs_dataset_path / "20240129_preds_confidence_howard_model.xlsx")